In [1]:
import echoshader
import xarray as xr
import panel as pn
import holoviews as hv
from holoviews import opts, streams
from holoviews.streams import PolyDraw, PolyEdit
from shapely.geometry import Polygon
import geopandas as gpd

import os
import numpy as np
import pandas as pd

hv.extension('bokeh')

In [2]:
DATA_DIR = "/Users/sashalai/echoshader/echoshader/test_data/concatenated_MVBS.nc"

ds = xr.open_mfdataset(
    DATA_DIR,
    data_vars=["minimal"],
    coords="minimal",
    combine="by_coords",
)

ds

<xarray.Dataset> Size: 4MB
Dimensions:            (channel: 4, echo_range: 150, ping_time: 875)
Coordinates:
  * channel            (channel) <U37 592B 'GPT  18 kHz 009072058c8d 1-1 ES18...
  * echo_range         (echo_range) float64 1kB 0.0 5.0 10.0 ... 740.0 745.0
  * ping_time          (ping_time) datetime64[ns] 7kB 2017-07-24T19:30:00 ......
    time1              (ping_time) datetime64[ns] 7kB dask.array<chunksize=(875,), meta=np.ndarray>
Data variables:
    Sv                 (channel, ping_time, echo_range) float64 4MB dask.array<chunksize=(4, 875, 150), meta=np.ndarray>
    frequency_nominal  (channel) float64 32B dask.array<chunksize=(4,), meta=np.ndarray>
    longitude          (ping_time) float64 7kB dask.array<chunksize=(875,), meta=np.ndarray>
    latitude           (ping_time) float64 7kB dask.array<chunksize=(875,), meta=np.ndarray>
Attributes:
    processing_software_name:     echopype
    processing_software_version:  0.7.1
    processing_time:              2023-05-30T17:40:45Z
    processing_function:          commongrid.compute_MVBS

In [3]:
# Create track
track = ds.eshader.track(tile='OpenTopoMap')

# Create echogram
channel = ds.coords['channel'].values[0]  # Get the first channel
eg = ds.eshader.echogram(
    channel=[channel],
    cmap="jet",
)()

# Create histogram
hist = ds.eshader.hist(
    bins = 20,
    overlay = True,
)

table = ds.eshader.table()

pn.Column(eg, hist, table)

BokehModel(combine_events=True, render_bundle={'docs_json': {'a7dc4b5f-6225-44fa-9250-43e337d9e8ac': {'version…

In [4]:
# Create empty Polygons element
poly = hv.Polygons([]).opts(
    opts.Polygons(
        fill_alpha=0.3,
        line_width=2,
        color='red',
        tools=['hover']
    )
)

# Create PolyDraw stream - this provides drawing capabilities
poly_draw = PolyDraw(source=poly, show_vertices=True, drag=True, num_objects = 50, 
                             vertex_style=dict(size=4, color='red'))


# Create PolyEdit stream - this allows editing existing polygons
poly_edit = PolyEdit(source=poly, vertex_style=dict(size=4, color='red'), shared=True)

# Overlay the echogram with the polygon element
overlay = eg * poly


In [5]:
dashboard = overlay.opts(
    opts.Polygons(fill_alpha=0.3, active_tools=['poly_draw', 'poly_edit'])
)

In [6]:
# Save path
SAVE_DIR = "/Users/sashalai/echoshader/docs/source/version_0.1.0/saved_data"

try:
    os.mkdir(SAVE_DIR)
    print(f"Directory '{SAVE_DIR}' created successfully.")
    
except FileExistsError:
    print(f"Directory '{SAVE_DIR}' already exists.")

start_time = str(ds['ping_time'][0].values)
end_time = str(ds['ping_time'][-1].values)

#file_name = start_time + '_' + end_time + '_' + str(channel) + '_regions.csv'
file_name = start_time + '_' + end_time + '_regions.csv'
save_path = os.path.join(SAVE_DIR, file_name)

Directory '/Users/sashalai/echoshader/docs/source/version_0.1.0/saved_data' already exists.


In [7]:
# Initialize the text output widget
text_output = pn.widgets.StaticText(value="")

In [8]:
# Create a callback that will extract polygon data
def export_callback(event):
    polygon_vertices = {
        'xs': poly_draw.data.get('xs', []),
        'ys': poly_draw.data.get('ys', [])
        }
    
    # Check if there are any polygons
    if not polygon_vertices['xs'] or not polygon_vertices['ys']:
        text_output.value = "No polygons to export. Please draw at least one polygon."
        return
    
    # Get only the polygons with more than 2 vertices
    valid_xs = [sublist for sublist in polygon_vertices['xs'] if len(sublist) > 2]
    valid_ys = [sublist for sublist in polygon_vertices['ys'] if len(sublist) > 2]

    if not valid_xs or not valid_ys:
        text_output.value = "No valid data extracted from polygons."
        return
    
    # Flatten the lists for the dataframe
    xs_flat = [x for sublist in valid_xs for x in sublist]
    ys_flat = [y for sublist in valid_ys for y in sublist]

    vertices_df = pd.DataFrame({'xs': xs_flat, 'ys': ys_flat})
    
    # Create region IDs 
    region_id = np.repeat(np.arange(len(valid_xs)), [len(x) for x in valid_xs])
    vertices_df['region_id'] = region_id

    vertices_df.to_csv(save_path, index=False)
    text_output.value = f"Saved data as: {file_name}"


In [9]:
# Export vertices button
export_button = pn.widgets.Button(name='Export Vertices', button_type='primary')
export_button.on_click(export_callback)


Watcher(inst=Button(button_type='primary', name='Export Vertices'), cls=<class 'panel.widgets.button.Button'>, fn=<function export_callback at 0x306ecc400>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [10]:
# Function to extract data from polygons
def extract_data_from_polygon(polygon_vertices, ds = ds, save=False):
    # Convert dataset into DataFrame and extract needed columns
    df = ds.to_dataframe().reset_index()
    df = df[['channel', 'ping_time', 'echo_range', 'Sv']]
    
    # Convert 'ping_time' to ns for plotting
    df['ping_time_numeric'] = pd.to_datetime(df['ping_time']).astype('int64') // 10**6

    # Create GeoDataFrame with points
    df['geometry'] = gpd.points_from_xy(df['ping_time_numeric'], df['echo_range'])
    gdf = gpd.GeoDataFrame(df, geometry='geometry')

    # Dictionary to store extracted data
    extracted_data = {}

    # Iterate over each polygon in the drawn data
    for i, (xs, ys) in enumerate(zip(polygon_vertices['xs'], polygon_vertices['ys'])):
        if len(xs) <= 2 or len(ys) <= 2:
            text_output.value = f"Skipping region {i+1}: needs at least 3 vertices"
            continue
            
        text_output.value = f"Processing region {i+1}..."
        
        # Create shapely Polygon
        polygon = Polygon(zip(xs, ys))
        
        # Get points inside the polygon using vectorized operations
        inside_mask = gdf['geometry'].within(polygon)
        
        if inside_mask.sum() == 0:
            text_output.value = f"No data points found in region {i+1}"
            continue
            
        filtered_df = gdf.loc[inside_mask, ['channel', 'ping_time', 'echo_range', 'Sv']]
        
        # Save to dictionary
        region_name = f"region_{i+1}"
        save_path = os.path.join(SAVE_DIR, f"{region_name}.csv")
        extracted_data[region_name] = filtered_df

        text_output.value = f"{region_name}: {inside_mask.sum()} points inside the selected polygon"

        # Optionally save each region to a CSV file
        if save:
            filtered_df.to_csv(save_path, index=False)
            text_output.value = f"Saved {region_name} at: {save_path}"

    return extracted_data


In [11]:
# Create histogram panel for displaying polygon histograms
histogram_panel = pn.Column(sizing_mode='stretch_width')

In [12]:
# Function to create and display histograms for polygon regions
def display_histograms_callback(event):
    # Clear previous histograms
    histogram_panel.clear()
    
    # Get polygon vertices
    polygon_vertices = {
        'xs': poly_draw.data.get('xs', []),
        'ys': poly_draw.data.get('ys', [])
    }
    
    # Check if there are any polygons
    if not polygon_vertices['xs'] or not polygon_vertices['ys']:
        text_output.value = "No polygons to analyze. Please draw at least one polygon."
        return
    
    # Extract data from polygons
    extracted_data = extract_data_from_polygon(polygon_vertices)
    
    # Check if the there is any valid data
    if not extracted_data:
        text_output.value = "No valid data extracted from polygons."
        return
    
    text_output.value = f"Creating histograms for {len(extracted_data)} regions..."
    
    # Create histograms for each region
    for region_name, region_data in extracted_data.items():
        if len(region_data) == 0:
            continue
            
        # Create histogram using HoloViews
        sv_values = region_data['Sv'].dropna().values
        if len(sv_values) == 0:
            continue
            
        hist_obj = hv.Histogram(np.histogram(sv_values, bins=20))
        hist_plot = hist_obj.opts(
            title=f"{channel} Sv Values - {region_name}",
            xlabel='Sv (dB)',
            ylabel='Frequency',
            height=250,
            width=500,
            color='blue'
        )
        
        # Add summary statistics 
        stats_text = f"""
            <b>Region:</b> {region_name}<br>
            <b>Count:</b> {len(sv_values)}<br>
            <b>Mean:</b> {np.mean(sv_values):.2f} dB<br>
            <b>Median:</b> {np.median(sv_values):.2f} dB<br>
            <b>Min:</b> {np.min(sv_values):.2f} dB<br>
            <b>Max:</b> {np.max(sv_values):.2f} dB<br>
            <b>Std Dev:</b> {np.std(sv_values):.2f} dB
        """
        
        stats_widget = pn.widgets.StaticText(value=stats_text)
        
        # Add to the histogram panel
        histogram_panel.append(pn.Row(hist_plot, stats_widget))
    
    text_output.value = f"Generated histograms for {len(extracted_data)} regions"

In [13]:
# Create button for displaying histograms
histogram_button = pn.widgets.Button(name='Display Histograms', button_type='primary')
histogram_button.on_click(display_histograms_callback)

Watcher(inst=Button(button_type='primary', name='Display Histograms'), cls=<class 'panel.widgets.button.Button'>, fn=<function display_histograms_callback at 0x306ecd1c0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

### PolyDraw

**Add patch/multi-line**

- Double tap to add the first vertex, then use tap to add each subsequent vertex, to finalize the draw action double tap to insert the final vertex or press the ESC key to stop drawing.

**Move patch/multi-line**

- Tap and drag an existing patch/multi-line; the point will be dropped once you let go of the mouse button.

**Delete patch/multi-line**

- Tap a patch/multi-line to select it then press BACKSPACE key while the mouse is within the plot area



### PolyEdit
**Show vertices**

- Long tap an existing patch or multi-line

**Add vertex**

- Double tap an existing vertex to select it, the tool will draw the next point, to add it tap in a new location.
- To finish editing and add a point double tap otherwise press the ESC key to cancel.

**Move vertex**

- Drag an existing vertex and let go of the mouse button to release it.

**Delete vertex**

- After selecting one or more vertices press BACKSPACE while the mouse cursor is within the plot area

In [14]:
# Initialize text and clear histograms on startup
text_output.value = "Draw polygons on the echogram"
histogram_panel.clear()

# Update layout to include the new histogram button and panel
layout = pn.Column(
    pn.Row(
        pn.Column(
            dashboard,
            pn.Row(export_button, histogram_button),
            text_output
        ),
    ),
    histogram_panel
)

# Display the layout
layout.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'3cc37244-5a4c-4ec4-8900-d1d1ff54ccf0': {'version…

In [15]:
polygon_vertices = poly_draw.data
polygon_vertices

{'xs': [[1501224870048.3525,
   1501215864248.9739,
   1501182910874.6387,
   1501202041255.0176],
  [1501448220696.6238, 1501469043058.0222, 1501434490123.9841]],
 'ys': [[154.7572169282511,
   75.8231589826233,
   101.44150259248876,
   174.6475616591928],
  [165.0669580297085, 278.39856887612103, 258.17978384248875]]}

### Save selected data into xarray datasets

In [20]:
def extract_data_from_polygon_xarray(polygon_vertices, ds, save = False):

    extracted_data = {}
        
    # Get all dimension names
    dim_names = list(ds.dims.keys())

    # Stack all dimensions into a single 'points' dimension
    stacked_ds = ds.stack(points=dim_names)

    # Create a temporary dataframe for the spatial operations
    coords_df = pd.DataFrame({
        'ping_time': stacked_ds.ping_time.values,
        'echo_range': stacked_ds.echo_range.values
    })

    # Convert ping_time to numeric for spatial operations
    coords_df['ping_time_numeric'] = pd.to_datetime(coords_df['ping_time']).astype('int64') // 10**6

    # Create geometries
    coords_df['geometry'] = gpd.points_from_xy(coords_df['ping_time_numeric'], coords_df['echo_range'])
    gdf = gpd.GeoDataFrame(coords_df, geometry='geometry')

    # Iterate over each polygon in the drawn data
    for i, (xs, ys) in enumerate(zip(polygon_vertices['xs'], polygon_vertices['ys'])):
        print(f"\nProcesing region {i+1}...")
        
        # Create shapely Polygon
        polygon = Polygon(zip(xs, ys))
        
        # Get points inside the polygon using vectorized operations
        inside_mask = gdf['geometry'].within(polygon)
        
        # Create a subset of the stacked dataset to preserves all variables and coordinates
        filtered_stacked_ds = stacked_ds.isel(points=inside_mask)
        
        filtered_stacked_ds = filtered_stacked_ds.drop_duplicates('points')
        
        # Unstack to restore original dimensions
        filtered_ds = filtered_stacked_ds.unstack()
        
        # Save to dictionary
        region_name = f"region_{i+1}"
        save_path = os.path.join(SAVE_DIR, f"{region_name}.nc")
        extracted_data[region_name] = filtered_ds
        
        print(f"{region_name}: {inside_mask.sum()} points inside the selected polygon")


        # Optionally save each region to NetCDF file
        if save:
            filtered_ds.to_netcdf(save_path)
            print(f"Saved at: {save_path}")
        
    return extracted_data

In [21]:
filtered_ds = extract_data_from_polygon_xarray(polygon_vertices, ds, save=True)


Procesing region 1...
region_1: 2168 points inside the selected polygon
Saved at: /Users/sashalai/echoshader/docs/source/version_0.1.0/saved_data/region_1.nc

Procesing region 2...
region_2: 1552 points inside the selected polygon
Saved at: /Users/sashalai/echoshader/docs/source/version_0.1.0/saved_data/region_2.nc


In [22]:
# Create histograms
def create_histograms_xarray(filtered_ds):
    histograms = []
    for i, region in enumerate(filtered_ds):
        hist = filtered_ds[region].eshader.hist(
            bins = 20,
            overlay = True,
        )
        histograms.append(hist)

    column = pn.Column(*histograms)
    
    return column

In [23]:
create_histograms_xarray(filtered_ds)

BokehModel(combine_events=True, render_bundle={'docs_json': {'81bd2d11-2170-42ba-ab05-dd250e280299': {'version…